In [ ]:
from pynq import Overlay, allocate
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
overlay = Overlay("fft32_overlay.bit")
overlay.download()

dma = overlay.axi_dma_0  # Replace with your DMA IP name
fft_ip = overlay.fft32_ip  # Replace with your FFT IP name

m = overlay.ip_inst

In [ ]:
N = 32
t = np.arange(N)
freq = 5  # Frequency in Hz
sample_rate = 32  # Samples per second
x_real = np.cos(2 * np.pi * freq * t / sample_rate)
x_imag = np.zeros(N)

in_buffer = allocate(shape=(N*2,), dtype=np.int16)
out_buffer = allocate(shape=(N*2,), dtype=np.int16)

# Convert to fixed-point representation (e.g., Q1.15 format)
scale_factor = 2**15
for i in range(N):
    in_buffer[2*i] = int(x_real[i] * scale_factor)
    in_buffer[2*i + 1] = int(x_imag[i] * scale_factor)


# # Set up the input and output buffers.  Use the `allocate` function since we need
# # to be able to find the physical addresses of these arrays later for the DMA.
# num_values = 400
# in_buffer = allocate(shape=(num_values,), dtype=np.uint32)
# out_buffer = allocate(shape=(num_values,), dtype=np.uint32)

# # Load the array with values: this happens on the CPU.
# for i in range(num_values):
#     in_buffer[i] = i

In [ ]:
# Set the multiplication constant
m.register_map.constant_r = 13

In [ ]:
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

# Start the FFT IP core
fft_ip.write(0x00, 0x01)  # Assuming control register at offset 0x00

dma.sendchannel.wait()
dma.recvchannel.wait()

# # Send and receive data using DMA.  This takes care of all the setup and transfer
# # and automatically takes the length of transfer based on the size used during `allocate`.
# d.sendchannel.transfer(in_buffer)
# d.recvchannel.transfer(out_buffer)
# d.sendchannel.wait()
# d.recvchannel.wait()

In [ ]:
# Optional to check the status of the DMA module
# d.sendchannel.idle, d.sendchannel.running, d.recvchannel.idle, d.recvchannel.running

In [ ]:
# Print the output for manual verification
out_buffer

In [ ]:
# Convert back to floating-point
y_real = np.zeros(N)
y_imag = np.zeros(N)
for i in range(N):
    y_real[i] = out_buffer[2*i] / scale_factor
    y_imag[i] = out_buffer[2*i + 1] / scale_factor

# Compute magnitude
y_mag = np.sqrt(y_real**2 + y_imag**2)

plt.stem(y_mag)
plt.title("FFT Magnitude")
plt.xlabel("Frequency Bin")
plt.ylabel("Magnitude")
plt.grid(True)
plt.show()
